In [1]:
import pandas as pd
import numpy as np

from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances

import torch

from mcce import MCCE

## FOR EACH DATA SET you have to adjust n below - 
## for adult and gmc, I use 100, 1000, 10000 and the size of the data set
## for compas, I use 100, 1000, 5000, and the size of the dat aset 

data_name = "adult"
data_name = 'give_me_some_credit'
data_name = 'compas'


K = 10000
n_test = 100
seed = 1

dataset = OnlineCatalog(data_name)

torch.manual_seed(0)
ml_model = MLModelCatalog(
        dataset, 
        model_type="ann", 
        load_online=False, 
        backend="pytorch"
        )

if data_name == 'adult':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=1024,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'give_me_some_credit':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=2048,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'compas':
    ml_model.train(
    learning_rate=0.002,
    epochs=25,
    batch_size=25,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )

# (2) Find unhappy customers and choose which ones to make counterfactuals for
factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:n_test]
# test_factual_inverse = dataset.inverse_transform(test_factual)

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/nr/samba/

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
balance on test set 0.8131345863037374, balance on test set 0.8191834089436163
Epoch 0/24
----------


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


train Loss: 0.5018 Acc: 0.7280

test Loss: 0.4022 Acc: 0.8192

Epoch 1/24
----------
train Loss: 0.4002 Acc: 0.8283

test Loss: 0.3704 Acc: 0.8471

Epoch 2/24
----------
train Loss: 0.3796 Acc: 0.8356

test Loss: 0.3557 Acc: 0.8419

Epoch 3/24
----------
train Loss: 0.3665 Acc: 0.8423

test Loss: 0.3405 Acc: 0.8477

Epoch 4/24
----------
train Loss: 0.3620 Acc: 0.8438

test Loss: 0.3483 Acc: 0.8464

Epoch 5/24
----------
train Loss: 0.3589 Acc: 0.8445

test Loss: 0.3409 Acc: 0.8542

Epoch 6/24
----------
train Loss: 0.3592 Acc: 0.8434

test Loss: 0.3361 Acc: 0.8555

Epoch 7/24
----------
train Loss: 0.3579 Acc: 0.8451

test Loss: 0.3358 Acc: 0.8529

Epoch 8/24
----------
train Loss: 0.3564 Acc: 0.8445

test Loss: 0.3367 Acc: 0.8548

Epoch 9/24
----------
train Loss: 0.3559 Acc: 0.8440

test Loss: 0.3383 Acc: 0.8516

Epoch 10/24
----------
train Loss: 0.3564 Acc: 0.8471

test Loss: 0.3534 Acc: 0.8542

Epoch 11/24
----------
train Loss: 0.3556 Acc: 0.8447

test Loss: 0.3376 Acc: 0.8535



In [2]:
y_col = dataset.target
features_and_response = dataset.df.columns
cont_feat = dataset.continuous
cat_feat = [x for x in features_and_response if x not in cont_feat] #  these have new names since encode_normalize_order_factuals()

if data_name == 'adult': 
    fixed_features = ['age', 'sex_Male']
    immutables = ['age', 'sex']
elif data_name == 'give_me_some_credit':
    fixed_features = ['age']
    immutables = ['age']
elif data_name == 'compas':
    fixed_features = ['age', 'sex_Male', 'race_Other']
    immutables = ['age', 'sex', 'race']

#  Create dtypes for MCCE()
dtypes = dict([(x, "float") for x in cont_feat])
for x in cat_feat:
    dtypes[x] = "category"
df = (dataset.df).astype(dtypes)

In [3]:
# print(df.dtypes)

In [4]:
# import random
# results = []

# dim = df.shape[0]
# random.seed(0)
# rows = random.sample(df.index.to_list(), dim)
# rows = np.sort(rows)

# df_subset = df.loc[rows]

# print(df.equals(df_subset))

# import time
# start = time.time()

# mcce = MCCE(fixed_features=fixed_features, immutables=immutables, model=ml_model, seed=1, continuous=cont_feat, categorical=cat_feat)

# mcce.fit(df_subset.drop(y_col, axis=1), dtypes)

# synth_df = mcce.generate(test_factual.drop(y_col, axis=1), k=K)

# mcce.postprocess(data=df_subset, synth=synth_df, test=test_factual, response=y_col, \
#     inverse_transform=dataset.inverse_transform, cutoff=0.5)

# timing = time.time() - start

# mcce.results_sparse['time (seconds)'] = timing

# results.append([mcce.results_sparse.L0.mean(), mcce.results_sparse.L2.mean(), mcce.results_sparse.feasibility.mean(), mcce.results_sparse.violation.mean(), mcce.results_sparse.shape[0], timing, n, 0])

In [5]:
# results2 = pd.DataFrame(results, columns=['L0', 'L2', 'feasibility', 'violation', 'NCE', 'timing', 's', 'n'])
# results2.mean()

## Loop through various subsets of data and train trees on the smaller subsets - 48832 is the full data set

In [8]:
import random 

# results = []
for n in [5000]: # 20000, 30000, 40000

    if n == 48832:
        random.seed(0)
        rows = random.sample(df.index.to_list(), n)
        rows = np.sort(rows)
        df_subset = df.loc[rows]
        print(df.equals(df_subset))

        import time
        start = time.time()

        mcce = MCCE(fixed_features=fixed_features, immutables=immutables, model=ml_model, seed=1,\
            continuous=cont_feat, categorical=cat_feat)

        mcce.fit(df_subset.drop(y_col, axis=1), dtypes)

        synth_df = mcce.generate(test_factual.drop(y_col, axis=1), k=K)

        # (4) Postprocess generated counterfactuals
        print("Postprocessing counterfactuals with MCCE...")
        mcce.postprocess(data=df, synth=synth_df, test=test_factual, response=y_col, \
            inverse_transform=dataset.inverse_transform, cutoff=0.5)

        timing = time.time() - start

        mcce.results_sparse['time (seconds)'] = timing

        results.append([mcce.results_sparse.L0.mean(), mcce.results_sparse.L2.mean(), mcce.results_sparse.feasibility.mean(), mcce.results_sparse.violation.mean(), mcce.results_sparse.shape[0], timing, n, 0])
    else:
        for s in range(5):

            random.seed(s)
            rows = random.sample(df.index.to_list(), n)
            rows = np.sort(rows)
            df_subset = df.loc[rows]
            print(df.equals(df_subset))

            import time
            start = time.time()

            mcce = MCCE(fixed_features=fixed_features, immutables=immutables, model=ml_model, seed=1,\
                continuous=cont_feat, categorical=cat_feat)

            mcce.fit(df_subset.drop(y_col, axis=1), dtypes)

            synth_df = mcce.generate(test_factual.drop(y_col, axis=1), k=K)

            mcce.postprocess(data=df, synth=synth_df, test=test_factual, response=y_col, \
                inverse_transform=dataset.inverse_transform, cutoff=0.5)

            timing = time.time() - start

            mcce.results_sparse['time (seconds)'] = timing

            results.append([mcce.results_sparse.L0.mean(), mcce.results_sparse.L2.mean(), mcce.results_sparse.feasibility.mean(), mcce.results_sparse.violation.mean(), mcce.results_sparse.shape[0], timing, n, s])


False


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [ ]:
data_name

In [7]:
results2 = pd.DataFrame(results, columns=['L0', 'L2', 'feasibility', 'violation', 'NCE', 'timing', 'Ntest', 'seed'])
results2.groupby('Ntest').mean()

,L0,L2,feasibility,violation,NCE,timing,seed
Ntest,,,,,,,
100,1.598,0.116386,0.0,0.0,100,2241.465002,2
1000,1.546,0.159257,0.0,0.0,100,2402.046115,2


In [ ]:
results2.to_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}_with_various_training_amounts.csv")

In [ ]:
# results3 = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}_with_various_training_amounts.csv", index_col=0)


In [ ]:
# results3.groupby('Ntest').mean()

In [ ]:
# mcce.results_sparse.to_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}.csv")

In [ ]:
# temp =pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/give_me_some_credit_mcce_results_k_10000_n_100_inverse_transform.csv", index_col=0)
# temp.loc[263]

In [ ]:
# dataset = OnlineCatalog("adult")

# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/adult_mcce_results_k_{K}_n_{n_test}.csv")

# dataset.inverse_transform(results.iloc[0:1])[['age', 'workclass', 'fnlwgt', 'education-num', 'marital-status', 'relationship', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']].iloc[0]


In [ ]:
# data_name = 'adult'
# K = 50000
# n_test = 100
# import pandas as pd
# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}_inverse_transform.csv")
# print(results.L0.mean())
# print(results.L1.mean())
# print(results.L2.mean())


In [ ]:
# # print(mcce.results_sparse.)
# print(results.L0.mean())
# print(results.L1.mean())
# print(results.L2.mean())
# print(results.feasibility.mean())
# print(results.violation.mean())
# print(results.success.mean())
# print(results.shape[0])

In [ ]:
# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_{K}_n_{n_test}.csv")

# results['data'] = data_name
# results['method'] = 'mcce'
# results.rename(columns={'violation': 'violations'}, inplace=True)

# preds = ml_model.predict_proba(results)
# new_preds = []
# for x in preds:
#     new_preds.append(x[1])
# results['prediction'] = new_preds
# results = dataset.inverse_transform(results)
# results.head(1)

# results['validity'] = np.where(np.asarray(new_preds) >= 0.5, 1, 0)

# results

In [ ]:
# pd.set_option('display.max_columns', None)

# dataset = OnlineCatalog("give_me_some_credit")

# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/give_me_some_credit_mcce_results_k_{K}_n_{n_test}.csv")

# results.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
# results.set_index(['index'], inplace=True)
# temp = results.sort_values([results.index.name]).iloc[0:1]
# dataset.inverse_transform(temp)[['age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']].iloc[0]


In [ ]:
# print(mcce.results_sparse.)
# print(mcce.results_sparse.L0.mean())
# print(mcce.results_sparse.L1.mean())
# print(mcce.results_sparse.L2.mean())
# results_sparse = mcce.results_sparse
# results_sparse.index.rename('index', inplace=True)
# results_sparse.groupby('index').size().sort_values(ascending=False)